# Using scan analyzer configs

Using scan analyzer configs makes wrapping multiple analysis types across multiple scans a straight forward process.

In [1]:
import logging
from geecs_data_utils import ScanPaths, ScanTag
from geecs_data_utils.config_roots import image_analysis_config, scan_analysis_config
from scan_analysis.config.config_loader import load_experiment_config
from scan_analysis.config.analyzer_factory import create_analyzer

# Quiet logs
for name in ("image_analysis", "scan_analysis", "geecs_data_utils"):
    logging.getLogger(name).setLevel(logging.ERROR)

# Point to config roots (or set env vars IMAGE_ANALYSIS_CONFIG_DIR / SCAN_ANALYSIS_CONFIG_DIR)
image_analysis_config.set_base_dir(ScanPaths.paths_config.image_analysis_configs_path)
scan_analysis_config.set_base_dir(ScanPaths.paths_config.scan_analysis_configs_path)

# Load experiment config (includes library/group resolution) and instantiate analyzers
cfg = load_experiment_config("undulator", config_dir=scan_analysis_config.base_dir)
analyzers = [create_analyzer(a) for a in cfg.get_analyzers_by_priority()]

In [2]:
# Choose recorded scans to process
tags = [
    ScanTag(year=2025, month=12, day=1, number=i, experiment="Undulator")
    for i in range(1, 3)
]

# Build work list sorted by (priority, scan number)
work = sorted(
    [(a.priority, t, a) for a in analyzers for t in tags],
    key=lambda x: (x[0], x[1].number),
)

In [4]:
analyzers

In [5]:
# Run analyzers
for _, tag, analyzer in work:
    analyzer.run_analysis(tag)